In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("db", "")
dbutils.widgets.text("model", "")
dbutils.widgets.text("run_id", "")
dbutils.widgets.text("user", "")
dbutils.widgets.text("num_nodes", "1")

In [0]:
model_class = "global" if "NeuralForecast" in dbutils.widgets.get("model") else "foundation" 
%pip install -r ../requirements-{model_class}.txt
dbutils.library.restartPython()

In [0]:
catalog = dbutils.widgets.get("catalog")
db = dbutils.widgets.get("db")
model = dbutils.widgets.get("model")
run_id = dbutils.widgets.get("run_id")
user = dbutils.widgets.get("user")
num_nodes = int(dbutils.widgets.get("num_nodes"))

In [0]:
from mmf_sa import run_forecast

run_forecast(
    spark=spark,
    train_data=f"{catalog}.{db}.m4_daily_train",
    scoring_data=f"{catalog}.{db}.m4_daily_train",
    scoring_output=f"{catalog}.{db}.daily_scoring_output",
    evaluation_output=f"{catalog}.{db}.daily_evaluation_output",
    model_output=f"{catalog}.{db}",
    group_id="unique_id",
    date_col="ds",
    target="y",
    freq="D",
    prediction_length=10,
    backtest_length=30,
    stride=10,
    metric="smape",
    train_predict_ratio=1,
    data_quality_check=False,
    resample=True,
    active_models=[model],
    experiment_path=f"/Users/{user}/mmf/m4_daily",
    use_case_name="m4_daily",
    run_id=run_id,
    accelerator="gpu",
    num_nodes=num_nodes,
)